In [5]:
import cv2
import random
import numpy as np
import os

VIDEO = "D:/STUDY/competition/competition/Intersection.mp4"
OUT_DIR = "D:/STUDY/competition/competition/Imgs"

count = 0
cap = cv2.VideoCapture(VIDEO)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.flip(frame,1)
    (h, w) = frame.shape
    center =  (w//2, h//2)
    M = cv2.getRotationMatrix2D(center, random.randint(0, 360), 1)
    frame = cv2.warpAffine(frame, M, (w, h))
    mean = 0
    std_dev = 10
    noise = np.random.normal(mean, std_dev, (h, w))
    frame = cv2.add(frame, noise.astype(np.uint8))

    filename = os.path.join(OUT_DIR + f"/frame_{count}.jpg")
    count += 1

    cv2.imwrite(filename, frame)

cap.release()
cv2.destroyAllWindows()

In [8]:
import torch
import cv2
import json

VIDEO = "D:/STUDY/competition/competition/Intersection.mp4"
OUT_DIR = "D:/STUDY/competition/competition/output.json"

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
cap = cv2.VideoCapture(VIDEO)
objects = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    detections = model(frame)
    for det in detections.xyxy[0]:
        class_id = int(det[5])
        class_name = model.names[class_id]
        confidence = float(det[4])
        bbox = det[:4].tolist()

        # Save the results to the list
        objects.append({
            'FrameID': cap.get(cv2.CAP_PROP_POS_FRAMES),
            'Class': class_name,
            'BoundingBox': bbox,
            'Confidence': confidence
        })

with open(OUT_DIR, 'w') as f:
    json.dump(objects, f)

Using cache found in C:\Users\DELL/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-6-24 Python-3.9.12 torch-2.0.1+cpu CPU

100.0%

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
